In [237]:
#libraries
import urllib2
from unidecode import unidecode
from bs4 import BeautifulSoup
import re
from unidecode import unidecode

In [2]:
#open episode lists page, put it in the beautiful soup
episodes_index = "http://www.clinic-duty.livejournal.com/12225.html"
episodes_index_pages = urllib2.urlopen(episodes_index)
episodes_index_soup = BeautifulSoup(episodes_index_pages, "lxml")

In [268]:
#get the links on the tables (it's on the tables)
table = episodes_index_soup.find('table')
episodes_url = []
#get those contain certain pattern
all_episodes_links = table.find_all('a', attrs={'href': re.compile("\/community\.livejournal\.com\/clinic_duty\/[0-9]+\.html")})
#get the href
for link in all_episodes_links:
    episodes_url.append(link.get("href"))

#remove first three (not what I'm seeking)
episodes_url = episodes_url[3:]

In [251]:
#seasons and how many episodes in each season
seasons = {"s1":22, "s2":24, "s3":24, "s4":16, "s5":24, "s6":22, "s7":23, "s8":22}

In [267]:
#clean unnecessary part
def clean(text):
    text = re.sub('<br/>','\\n',text)
    text = re.sub('Written by:.*?\\n','',text)
    text = re.sub('Originally Aired:.*?\\n','',text)
    text = re.sub('Directed by:.*?\\n','',text)
    text = re.sub('Transcribed by:.*?\\n','',text)
    text = re.sub('Betaed by:.*?\\n','',text)
    text = re.sub('DISCLAIMER:.*?\\n','',text)
    text = re.sub('\[.*?\]','',text)
    text = re.sub('\(.*?\)','',text)
    text = re.sub('<.*?>','',text)
    return text

#put in on txt named after its season and episode number
counter = 0
for season,eps in sorted(seasons.iteritems()):
    for i in range(1,int(eps)+1):
        eps_url = episodes_url[counter]
        eps_page = urllib2.urlopen(eps_url)
        eps_soup = BeautifulSoup(eps_page, "lxml")
        
        eps_entrytext = eps_soup.findAll("div", { "class" : "entryText" })
        clean_eps = clean(str(eps_entrytext[0]))
        
        dialogue_pattern = re.compile('\\n(.*?)\\n')
        dialogues = dialogue_pattern.findall(clean_eps)
        dialogues = [unidecode(s.decode('utf8')) for s in dialogues if s]
        
        output_name = season + "e" + str(i) + ".txt"
        with open(output_name, "w") as output:
            for dialogue in dialogues:
                output.write("%s\n" % dialogue)
                
        counter = counter+1                  